In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier

# Loading the data

In [2]:
#import the data
print('loding the data')
train_df=pd.read_csv('train.csv')# , index_col = 0 we want to use first column in csv as index#header none means header is treated as a row
test_df=pd.read_csv('test.csv')

loding the data


# Visualization

In [3]:
#train_df.head()
#train_df.info()
#train_df.describe()
#train_df['id']
#train_df.columns
print("Train rows and columns : ", train_df.shape)
print("Test rows and columns : ", test_df.shape)
num_train = train_df.shape[0] # no of entries

Train rows and columns :  (61878, 95)
Test rows and columns :  (144368, 94)


### Datatypes

In [4]:
dtype_df = train_df.dtypes.reset_index()
#dtype_df
dtype_df.columns = ["Count", "Column Type"]
#dtype_df
dtype_df.groupby("Column Type").aggregate('count').reset_index() #counting no. of rows in different datatypes 


,Column Type,Count
0,int64,94
1,object,1


# Preprocessing

### Missing values

In [5]:
missing_df = train_df.isnull().sum(axis=0).reset_index() #gives how many null values are present in each column
#missing_df # reset index attaches a new index column starting with 0
missing_df.columns = ['column_name', 'missing_count'] #naming the columns
#missing_df
missing_df = missing_df[missing_df['missing_count']>0]# printing  missing values 
#missing_df # printing  missing values 
missing_df = missing_df.sort_values(by='missing_count')
missing_df

# no missing values in this data set

,column_name,missing_count


### Constant values

In [6]:
unique_df = train_df.nunique().reset_index() #returns no. of unique elements in each column
unique_df.columns = ["col_name", "unique_count"] #naming the columns

constant_df = unique_df[unique_df["unique_count"]==1] #if column has only 1 value
constant_df.shape #no. of such rows
#constant_df # printing such rows
#str(constant_df.col_name.tolist())

#no constant columns in this dataset

(0, 2)

### splitting into X and y

In [7]:
y_train=train_df['target']
X_train=train_df.drop('target', 1)

### Label Encoding 

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() #Encode labels with value between 0 and n_classes-1.

le.fit(y_train.values.tolist()) # converted df to array to list
label=le.transform(y_train)
print(list(le.classes_))
print(label)

['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
[0 0 0 ... 8 8 8]


### train-test split

In [9]:
#train test split into X and y
from sklearn.model_selection import train_test_split

print('train, dev, test split')
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1) #train 80, test 20


train, dev, test split


# Model Building

### LogisticRegression

In [10]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print('LogisticRegression LogLoss {score}'.format(score=log_loss(y_test, logreg.predict_proba(X_test))))

LogisticRegression LogLoss 0.6898310737915901


### RandomForestClassifier

In [11]:
rfc = RandomForestClassifier(n_estimators=50, random_state=1337, n_jobs=-1)
rfc.fit(X_train, y_train)
print('RFC LogLoss {score}'.format(score=log_loss(y_test, rfc.predict_proba(X_test))))

RFC LogLoss 0.15299864174793554


### XGBClassifier

In [12]:
xg = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=.8,subsample=.2,
       gamma=0,learning_rate=0.1,max_delta_step= 4,max_depth=5,
       missing=None,n_estimators= 400,nthread=2,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27,silent=1)
xg.fit(X_train, y_train)
XG_predictions = xg.predict(X_test)
print('XGBclassifier LogLoss {score}'.format(score=log_loss(y_test, xg.predict_proba(X_test))))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


XGBclassifier LogLoss 0.0006793786808005249


In [13]:
test_df=pd.read_csv('test.csv', index_col=None)
test_df.head()
test_predictions = xg.predict(test_df)
submission = pd.DataFrame({'id': test_df.id, 'target' : test_predictions} )
submission.to_csv('submission.csv', index=False)
submission.head()

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,id,target
0,1,Class_1
1,2,Class_1
2,3,Class_1
3,4,Class_1
4,5,Class_1
